<a href="https://colab.research.google.com/github/rrlage/NLPyPort/blob/master/TestesIn%C3%ADcio_29_05__v07_06_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import os
import pathlib as Path
import pickle
import re

from huggingface_hub import hf_hub_download


import torch
import nltk
import nltk.data
import numpy as np


import pandas as pd

from nltk import Tree
from nltk.chunk import RegexpParser, conlltags2tree, tree2conlltags
from nltk.corpus import conll2000, floresta
from nltk.draw import TreeWidget
from nltk.draw.tree import TreeView
from nltk.draw.util import CanvasFrame
from nltk.grammar import *
from nltk.parse.chart import AbstractChartRule, Chart
from nltk.tag import DefaultTagger, UnigramTagger, pos_tag


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!pip install click
!pip install xmltodict
!pip install unidecode
!pip install python-crfsuite
!pip install spacy-lookups-data
!pip install tabulate
nltk.download('floresta')
nltk.download('punkt')

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
from google.colab import files

In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks')

In [ ]:
import gensim
#import ghostscript
import nltk
import nltk.data
import spacy
from NLPyPort.CRF import *
from NLPyPort.LemPyPort import *
from NLPyPort.TagPyPort import *
from NLPyPort.text import *
from NLPyPort.TokPyPort import *

from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile
from nltk import ne_chunk, sent_tokenize, word_tokenize
from nltk.corpus import floresta, mac_morpho, machado
from nltk.tokenize import LineTokenizer
from nltk.tokenize import TreebankWordTokenizer as twt
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.tokenize.util import string_span_tokenize
from spacy import *
from spacy.attrs import ENT_TYPE, IS_ALPHA, LOWER, POS
from spacy.lang.pt.examples import sentences
from spacy.pipeline.senter import DEFAULT_SENTER_MODEL
from spacy.tokens import Doc, Span

In [ ]:
!pip install spacy

In [ ]:
from spacy.training import Example
import spacy.cli
from spacy.tokens import Doc, DocBin
import random
from tqdm import tqdm
import json
spacy.cli.download("pt_core_news_lg")
#spacy.cli.download("en_core_web_lg")
#spacy.cli.download("en_core_web_sm")

In [ ]:
uploaded_files = files.upload()

In [ ]:
import locale
import torch
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
spacy.__version__

In [ ]:
!pip install cupy-cuda12x

In [ ]:
spacy.prefer_gpu()
nlp = spacy.load("pt_core_news_lg")
nlp.max_length = 10000000000
#PdfDir = "/Users/MNR/knime_workspace_0/Doutorado Unicamp_Experimental/KNIME/arquivos de entrada/PDFs_PCK_PTBR/Treino/"
#CsvDir = "/Users/MNR/teste/CsvPCK/"
#files = glob.glob(CsvDir + "*.csv", recursive=True)

In [ ]:
nlp = spacy.load("en_core_web_lg")
#nlp = spacy.load("en_core_web_lg")
nlp.max_length = 1000000000

In [ ]:
dfL = pd.read_csv("csv_pck.csv", sep=",")

In [ ]:
listDocs = []
docs = list(nlp.pipe(listDocs))
for i in range(0, len(dfL['Conteúdo'])):
    art = nlp(dfL['Conteúdo'][i])
    docs.append(art)
c_doc = Doc.from_docs(docs)

In [ ]:
listDocs = []
docs = list(nlp.pipe(listDocs))
dfT['line_text'] = dfT['line_text'].apply(str)
for i in range(0, len(dfT)):
    art = nlp(dfT['line_text'][i])
    docs.append(art)
c_doc = Doc.from_docs(docs)

In [ ]:
i = 0
for i in range(0, len(data)):
    print(nlp(data[i][0]))

In [ ]:
dfL['Conteúdo'] = dfL['Conteúdo'].str.replace("  ", " ")

In [ ]:
import blis

In [ ]:
for doc in docs:
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
dfL['Conteúdo'] = dfL['Conteúdo'].str.replace(r'[\W_]+', ' ', regex=True).str.strip()

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
# Define a function to create spaCy DocBin objects from the annotated data
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('pt')
  db = DocBin()

  # Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [ ]:
# Split the annotated data into training and testing sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.2)

# Display the number of items in the training and testing sets
len(train), len(test)

# Open a file to log errors during annotation processing
file = open('/content/train_file.txt','w')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk('/content/train.spacy')

db = get_spacy_doc(file, test)
db.to_disk('/content/dev.spacy')

# Close the error log file
file.close()

In [ ]:
listSents = []
Sents = list(nlp.pipe(listSents))

for j in range(0, len(docs)):
    for i in range(0, len(list(docs[j].sents))):
        arts = nlp(str(list(docs[j].sents)[i]))
        Sents.append(arts)
    c_Sent = Doc.from_docs(Sents)

In [ ]:
import spacy

In [ ]:
def train_spacy(data, iterations, nlp):  # <-- Add model as nlp parameter
    data = dataset
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.add_pipe('ner')
        nlp.add_pipe('sentencizer')
    else:
        ner = nlp.get_pipe('ner')
        nlp.add_pipe('sentencizer')

    # add labels
    for _, annotations in data:
         for ent in annotations.get('entities'):
             ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            random.shuffle(data)
            losses = {}
            for text, annotations in data:
                nlp.update(
                    examples,  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp
nlp = spacy.load("pt_core_news_lg")  # create blank Language class
start_training = train_spacy(dataset, 300, nlp)

In [ ]:
i = 0
for i in range(0, len(data)):
    data[i]['data']['text'] = data[i]['data']['text'].replace('\t', '')

In [ ]:
for i in range(0, len(data)):
    j = 0
    for j in range(0, len(data[i]['annotations'][0]['result'])):
        if 'id' in data[i]['annotations'][0]['result'][j]:
            data[i]['annotations'][0]['result'][j].pop('id')
        if 'origin' in data[i]['annotations'][0]['result'][j]:
            data[i]['annotations'][0]['result'][j].pop('origin')


In [ ]:
i = 0
j = 0
dataset = []
d = {}
for i in range(0, len(data)):
    for j in range(0, len(data[i]['annotations'][0]['result'])):
      if 'value' in data[i]['annotations'][0]['result'][j]:
        d = {'entities': [(data[i]['annotations'][0]['result'][j]['value']['start'], data[i]['annotations'][0]['result'][j]['value']['end'], data[i]['annotations'][0]['result'][j]['value']['labels'][0])]}
      else:
        pass
    dataset.append([data[i]['data']['text'], d])

In [ ]:
i = 0
j = 0
examples = []
for i in range(0, len(dataset)):
    example = Example.from_dict(nlp_lg.make_doc(dataset[i][0]), dataset[i][1])
    examples.append(example)

In [ ]:
annot

In [ ]:
type(tasks[0]['predictions'][0]['result'][1]['value']['labels'])

In [ ]:
ask['predictions'][0]['model_version']

In [ ]:
import json
from collections import defaultdict

tasks = json.load(open('annotations.json', 'r', encoding='utf-8'))
model_hits = defaultdict(int)

for task in tasks:
    annotation_result = task['annotations'][0]['result']
    for r in annotation_result:
        r.pop('id')
    for r in annotation_result:
        r.pop('origin')
    for i in range(0,task['predictions']:
        model_hits[task['predictions'][0]['model_version']] += int(task['predictions'][0]['result'] == annotation_result)

num_task = len(tasks)
for model_name, num_hits in model_hits.items():
    acc = num_hits / num_task
    print(f'Accuracy for {model_name}: {acc:.2f}%')

In [ ]:
import json
from collections import defaultdict

tasks = json.load(open('annotations.json'))
model_hits = defaultdict(int)

for task in tasks:
    annotation_result = task['annotations'][0]['result']
    for r in annotation_result:
        r.pop('id')
    for prediction in task['predictions']:
        model_hits[prediction['model_version']] += int(prediction['result'] == annotation_result)

num_task = len(tasks)
for model_name, num_hits in model_hits.items():
    acc = num_hits / num_task
    print(f'Accuracy for {model_name}: {acc:.2f}%')

In [ ]:
tasks = json.load(open('annotations.json', encoding='utf-8'))


i = 0
for i in range(0, len(tasks)):
  j = 0
  for j in range(0, len(tasks[i]['annotations'][0]['result'])):
    tasks[i]['annotations'][0]['result'][j].pop('id')
    tasks[i]['annotations'][0]['result'][j].pop('origin')
    a = tasks[i]['annotations'][0]['result']
    b = tasks[i]['annotations'][0]['prediction']['result']
    x = len(tasks[i]['annotations'][0]['result'])
    y = len(tasks[i]['annotations'][0]['prediction']['result'])
    model_name = tasks[i]['annotations'][0]['prediction']['model_version']
  num_hits = len(list(zip(a, b)))
  num_task = x if x > y else y
  acc = num_hits / num_task
  print(str(i) + f'Accuracy for {model_name}: {acc:.2f}%')

In [ ]:
dataset

In [ ]:
for j in range(0, len(tasks[i]['annotations'][0]['result'][j])):
  tasks[i]['annotations'][0]['result'][j].pop('id')
  tasks[i]['annotations'][0]['result'][j].pop('origin')

In [ ]:
a = tasks[31]['annotations'][0]['result']
b = tasks[31]['annotations'][0]['prediction']['result']
pairs = zip(a, b)
any(x == y for x, y in pairs)

In [ ]:
list(zip(b, a))

In [ ]:
x = len(tasks[10]['annotations'][0]['result'])
y = len(tasks[10]['annotations'][0]['prediction']['result'])
num_task = x if x > y else y

In [ ]:
a

In [ ]:
b

In [ ]:
b = tasks[15]['annotations'][0]['prediction']['result']

In [ ]:
tasks[2]['annotations'][0]['prediction']['model_version']

In [ ]:
tasks[2]['annotations'][0]['result'][0] == tasks[2]['annotations'][0]['prediction']['result'][0]

In [ ]:
tasks[33]['annotations'][0]['result'][0]  == annotation_result[0]

In [ ]:
task['annotations'][0]['result'][0]

In [ ]:
tasks[0]['annotations'][0]['result'][0] == annotation_result[0]

In [ ]:

#task['annotations'][0]['result']
#prediction['result'][0]
predictions[0]['result']

In [ ]:
annotation_result[0]

In [ ]:
  for prediction in predictions:
    print(prediction['model_version'])

In [ ]:
annotation_result

#== prediction['result'][0]

#prediction['result'][0]

In [ ]:
def Convert(tup, di):
    di = dict(tup)
    return di
for pred in prediction['result']:
  tups += pred['result']

prediction['result'] = (Convert(tups, dictionary))

In [ ]:
def Convert(tup, di):
    di = dict(tup)
    return di
tups = prediction['result'][0]
dictionary = {}
print (Convert(tups, dictionary))

In [ ]:
def Convert(tup, di):
    di = dict(tup)
    return di
tups = annotation_result
dictionary = {}
print (Convert(tups, dictionary))

In [ ]:
import json
from collections import defaultdict
tasks = json.load(open('annotations.json'))
model_hits = defaultdict(int)

for task in tasks:
    annotation_result = task['annotations'][0]['result']
    for r in annotation_result:
        r.pop('id')
        print(annotation_result)
    for prediction in task['predictions']:
        print(prediction)

In [ ]:
task['annotations'][0]['result']

In [ ]:
for annotation in annotation_result:
    print(annotation)

In [ ]:
dir(doc)

In [ ]:
doc = nlp_lg.make_doc(annot[0])
ents = []
i = 0
for i in range(0, len(annot[1]['entities'])):
   span = doc.char_span(annot[1]['entities'][i][0], annot[1]['entities'][i][1], label=annot[1]['entities'][i][2], alignment_mode="expand")
   ents.append(span)
doc.ents = ents

In [ ]:
len(annot[1]['entities'])

In [ ]:
db = DocBin()
dataset = dataset
i = 0
for i in range(0, len(dataset)):
    doc = nlp.make_doc(dataset[i][0])
    ents = []
    span = doc.char_span(dataset[i][1]['entities'][0], dataset[i][1]['entities'][1], label=dataset[i][1]['entities'][2], alignment_mode="contract")
    ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk(path)
convert("train.spacy", TRAIN_DATA)
convert("dev.spacy", DEV_DATA)

In [ ]:

db = DocBin()
for text, annotations in dataset:
    doc = nlp(text)
    ents = []
    for annotations['entities'] in annotations:
        span = doc.char_span(annotations['entities'][0][0], annotations['entities'][0][1], label=annotations['entities'][0][2])
        ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./train.spacy")

In [ ]:
for text, annotations in dataset:
  print(text)
  print(annotations['entities'][0])


In [ ]:
annotations['entities'][0][0]

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
!python -m spacy init config --help

In [ ]:
!python -m spacy init config config.cfg --lang pt --pipeline ner --optimize accuracy --force

In [ ]:
!python -m spacy train config.cfg --output /content/output --paths.train ./train.spacy --paths.dev ./dev.spacy

In [ ]:
# Requires Python >=3.8
#!pip install label-studio

# Start the server at http://localhost:8080
label-studio

In [ ]:
!label-studio

In [ ]:
dtr = {}
i = 0
for i in range(0, len(TRAIN_TREM)):
 dtr.update({
    TRAIN_TREM[i][0][TRAIN_TREM[i][1]['entities'][0][0]:TRAIN_TREM[i][1]['entities'][0][1]]:
    TRAIN_TREM[i][1]['entities'][0][2]
})


In [ ]:
dfr = pd.DataFrame.from_dict(dtr, orient="index")

In [ ]:
dfr.to_csv("dfr.csv", sep=",")

In [ ]:
eval(pdD['entities'][0], engine='python')

In [ ]:
TRAIN_T = []
TRAIN_A = []
TRAIN_TREM = []
i=0
for i in range(0, len(pdD)):
            TRAIN_TREM.append(
                [
                    pdD['Texto'][i],
                    {
                        'entities': [
                            eval(pdD['entities'][i], engine='python')
                        ]
                    },
                ]
            )

NOVO TESTE BASEADO NO ARTIGO DISPONÍVEL EM: https://www.getyourguide.careers/posts/building-custom-ner-models-with-spacy-to-enhance-travel-product-inventory

INÍCIO EM 29/05/2024



In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [1]:
!pip install -U pip setuptools wheel
!pip install -U 'spacy[cuda12x]'
!python -m spacy download pt_core_news_lg

  Using cached https://github.com/explosion/spacy-models/releases/download/pt_core_news_lg-3.7.0/pt_core_news_lg-3.7.0-py3-none-any.whl (568.2 MB)
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
!pip install spacy[lookups,transformers]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

In [4]:
import spacy
spacy.prefer_gpu()
from spacy.tokens import DocBin, Doc, Span
from spacy.training.example import Example
#spacy.cli.download("pt_core_news_lg")
nlp_lg = spacy.load("pt_core_news_lg")

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [5]:
import spacy.cli.train
from spacy.cli.train import train

In [6]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
from google.colab import files

Mounted at /content/drive


In [7]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install jsonlines

In [9]:
uploaded_files = files.upload()

Saving annotNER7_min.json to annotNER7_min.json
Saving annotations (7).json to annotations (7).json
Saving annotNER6.json to annotNER6.json
Saving csv_pck_rmpunc.csv to csv_pck_rmpunc.csv
Saving csv_pck.csv to csv_pck.csv


In [10]:
dfL = pd.read_csv("csv_pck.csv", sep=",")

In [11]:
dfLr = pd.read_csv("csv_pck_rmpunc.csv", sep=",")

In [12]:
import json
import jsonlines

f = open("annotNER6.json", encoding='utf8')

data = json.load(f)

In [13]:
f = open("annotations (7).json", encoding='utf8')

dataA = json.load(f)

In [14]:
f = open("annotNER7_min.json", encoding='utf8')

datam = json.load(f)

In [15]:
i = 0
j = 0
for i  in range(0, len(datam)):
  for j in range(0, len(datam[i]['label'])):
    datam[i]['label'][j].pop('text')
    datam[i]['text'] = datam[i]['text'].replace('\t', '')


In [16]:
from itertools import chain
listen = []
for i in range(0, len(datam)):
  listen.append(len(datam[i]['label']))
##ACHO QUE NEM PRECISA!!!!!

In [17]:
sum(listen)

823

In [18]:
i = 0
j = 0
classes = dataA['classes']
annotations = []
annotations.append({'classes': classes})
for i in range(0, len(datam)):
  annotations.append(datam[i]['text'])
  entities = []
  for j in range(0, len(datam[i]['label'])):
    entities.append((datam[i]['label'][j]['start'], datam[i]['label'][j]['end'], datam[i]['label'][j]['labels'][0]))
  annotations.append({'entities': entities})

In [21]:
annot = annotations[1:]

{'classes': ['PER',
  'ORG',
  'LOC',
  'EVENTO',
  'CONCEITO',
  'COISA',
  'DATA',
  'REFERÊNCIA',
  'REFLEXÃO',
  'TERMO',
  'ATITUDE',
  'CENÁRIO',
  'TÉCNICA']}

In [22]:
annotation = []
i = 0
for i in range(0, len(annot)):
  annotation.append(annot[2*i:2*i+2])

In [23]:
annotation = annotation[0:102] #ENCONTRAR O CAMINHO MAIS ADEQUADO

In [ ]:
annotation[101]

In [24]:
annot[107]['entities'] = annot[107]['entities'][1:] #Marquei o mesmo segmento com duas etiquetas diferentes. Por isso a correção

In [29]:
i = 0
j = 0
examples = []
for i in range(0, len(annotation)):
    example = Example.from_dict(nlp_lg.make_doc(annotation[i][0]), annotation[i][1])
    examples.append(example)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Função hidrocarbonetos são compostos constituídos ..." with entities "[(0, 22, 'CONCEITO'), (27, 36, 'TERMO'), (61, 68, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


In [44]:
annotation[0][1]

{'entities': [(0, 22, 'CONCEITO'),
  (27, 36, 'TERMO'),
  (61, 68, 'COISA'),
  (71, 81, 'COISA'),
  (92, 109, 'CONCEITO'),
  (139, 142, 'CONCEITO')]}

In [ ]:
spacy.training.offsets_to_biluo_tags(nlp_lg.make_doc(annotation[1][0]), annotation[1][1]['entities'])

In [ ]:
i = 0
j = 0
examples = []
for i in range(0, len(annotation)):
    spacy.training.offsets_to_biluo_tags(nlp_lg.make_doc(annotation[i][0]), annotation[i][1]['entities'])

In [ ]:
TRAIN_DATA = examples[:60]
DEV_DATA = examples[60:]

In [ ]:
TRAIN_DATA[0]

In [ ]:
import spacy
from spacy.tokens import DocBin

db = DocBin()
# Iterate over examples and add their Doc objects to the DocBin
for example in TRAIN_DATA:
    db.add(example.reference)
db.to_disk("train.spacy")

db = DocBin()
for example in DEV_DATA:
    db.add(example.reference)
db.to_disk("dev.spacy")

In [ ]:
entities

In [ ]:
len(annotation[:60])

In [ ]:
db = DocBin()
i = 0
j = 0
#spacy.prefer_gpu()
nlp = spacy.blank("pt")
while i <= len(annotation[:60]):
    doc = nlp_lg.make_doc(annotation[i][0])
    ents = []
    while j <= len(annotation[i][1]['entities']):
        span = doc.char_span(annotation[i][1]['entities'][j][0], annotation[i][1]['entities'][j][1], label=annotation[i][1]['entities'][j][2], alignment_mode="expand")
        ents.append(span)
        j += 1
    doc.ents = ents
    db.add(doc)
    i += 1
db.to_disk("./train.spacy")

In [50]:
db = DocBin()
i = 0
j = 0
spacy.prefer_gpu()
nlp = spacy.blank("pt")
for i in range(0, len(annot[:186])):
  if i == 0 or i % 2 == 0:
    doc = nlp_lg.make_doc(annot[i])
    ents = []
  else:
    for j in range(0, len(annot[i]['entities'])):
        span = doc.char_span(annot[i]['entities'][j][0], annot[i]['entities'][j][1], label=annot[i]['entities'][j][2], alignment_mode="expand")
        ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./train.spacy")


In [ ]:
annot[len(annot[:122])]

In [51]:
db = DocBin()
i = 0
j = 0
spacy.prefer_gpu()
nlp = spacy.blank("pt")
for i in range(0, len(annot)):
  if i == 0 or i % 2 == 0:
    doc = nlp_lg.make_doc(annot[i])
    ents = []
  else:
    for j in range(0, len(annot[i]['entities'])):
        span = doc.char_span(annot[i]['entities'][j][0], annot[i]['entities'][j][1], label=annot[i]['entities'][j][2], alignment_mode="expand")
        ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./dev.spacy")


In [52]:
!python -m spacy init fill-config base_config.cfg config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [53]:
!python -m spacy train config.cfg --output /content/output --paths.train ./train.spacy --paths.dev ./dev.spacy --gpu-id 0

✔ Created output directory: /content/output
ℹ Saving to output directory: /content/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
tokenizer_config.json: 100% 43.0/43.0 [00:

In [54]:
!pip install spacy-transformers
import spacy_transformers
nlp_ner = spacy.load("/content/output/model-best")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [62]:
listDocs = []
docs = list(nlp_ner.pipe(listDocs))
for i in range(0, len(dfL['Conteúdo'])):
    art = nlp_ner(dfL['Conteúdo'][i])
    docs.append(art)
c_doc = Doc.from_docs(docs)

<ipython-input-62-f7c6d9adb82e>:6: UserWarning: [W101] Skipping Doc custom extension 'trf_data' while merging docs.
  c_doc = Doc.from_docs(docs)


In [63]:
for doc in docs:
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)

Função hidrocarbonetos 0 22 CONCEITO
compostos 27 36 TERMO
carbono 61 68 COISA
hidrogênio 71 81 COISA
fórmula molecular 92 109 CONCEITO
C-H. 139 143 CONCEITO
a gente 31 38 PER
em casa 43 50 LOC
a professora pergunta aos alunos 53 85 EVENTO
bolinhas brancas fedidas 107 131 COISA
os alunos respondem 134 153 EVENTO
Naftalina 156 165 COISA
a professora prossegue à explicação 168 203 EVENTO
composto aromático 222 240 TERMO
anéis conjugados 255 271 CONCEITO
a professora fala apresentando a estrutura da naftalina no quadro 274 339 EVENTO
estrutura planar 421 437 TERMO
conjugando um com o outro 439 464 CONCEITO
ressonar 482 490 CONCEITO
sublima 616 623 TERMO
estado sólido 636 649 CONCEITO
estado gasoso 664 677 CONCEITO
cancerígeno 687 698 TERMO
é extremamente necessário ter alternativas de ensino 0 52 REFLEXÃO
a cada tempo é mais difícil ter atenção e dedicação 66 117 TERMO
alunos 128 134 PER
estamos ampliando as possibilidades de tornar nossas aulas melhores e mais atrativas 184 268 TERMO
Pel


APARENTE BLOCO PROMISSOR JÁ EM DESENVOLVIMENTO DESDE 29/05. OTIMIZADO EM 27/06/2024

In [ ]:
TRAIN_DATA = dataset[:60]
DEV_DATA = dataset[60:]

In [ ]:
annot[:100]

In [ ]:
annot[100:]

In [ ]:
vocab = nlp.vocab
docs = db.get_docs(vocab)



In [ ]:
for doc in docs:
    print(doc.text, doc.ents)

In [ ]:
datam[99]

In [ ]:
i = 0
j = 0
for i in range(0, len(datam)):
  print(len(datam[i]['label']))

In [ ]:
len(datam[0]['label'])

In [ ]:
datam[1]['label']

In [ ]:
entities[6:23]

In [ ]:
entities[sum(listen[0:2]) : sum(listen[0:3])]

In [ ]:
i = 0
j = 0
tags = []
for i in range(0, len(datam)):
  for j in range(0, len(datam[i]['label'])):
    tags.append([datam[i]['label'][j]['start'], datam[i]['label'][j]['end'], datam[i]['label'][j]['labels'][0]])

In [ ]:
i = 0
for i  in range(0, len(datam)):
  datam[i].pop('id')
  datam[i].pop('annotator')
  datam[i].pop('annotation_id')
  datam[i].pop('created_at')
  datam[i].pop('updated_at')
  datam[i].pop('lead_time')

In [ ]:
with jsonlines.open('annotNER6.json') as fl:
    datal = [obj for obj in fl]


In [ ]:
a = []
for item in datal:
  a.append(item)

In [ ]:
for i in range(0, len(a[0])):
  print(len(a[0][i]['annotations'][0]['result']))

In [ ]:
a[0][99]['annotations']

In [ ]:
rem_lista0 = ['id', 'completed_by', 'was_cancelled', 'ground_truth', 'created_at', 'updated_at', 'draft_created_at', 'lead_time', 'prediction', 'result_count', 'unique_id', 'import_id', 'last_action', 'task', 'project', 'updated_by', 'parent_prediction', 'parent_annotation', 'last_created_by']

i = 0
for i in range(0, len(a[0])):
   for key in rem_lista0:
     del a[0][i]['annotations'][0][key]

In [ ]:
a[0][0]['annotations']

In [ ]:
a[0][2]['annotations'][0]['result']

In [ ]:
for i in range(0, len(a[0])):
   print(len(a[0][i]['annotations'][0]['result']))

In [ ]:
rem_lista1 = ['id', 'from_name', 'to_name', 'type', 'origin']
i = 0
for i in range(0, len(a[0])):
  j = 0
  for j in range(0, len(a[0][i]['annotations'][0]['result'])):
      for key in rem_lista1:
           del a[0][i]['annotations'][0]['result'][j][key]


In [ ]:
i = 0
ldt = []
for i in range(0, len(data)):
    for annotations in data[i]['annotations']:
        ldt.append(annotations['result'])

In [ ]:
dataA['annotations']

In [ ]:
ldt[0][0]['value']['labels'][0]

In [ ]:
ltt[0][27:36]

In [ ]:
i = 0
for i in range(0, len(ltt)):
    ltt[i] = ltt[i].replace('\t', '')


In [ ]:
data[0]['data']['text']

In [ ]:
i = 0
ltt = []
for i in range(0, len(data)):
    ltt.append(data[i]['data']['text'])


In [ ]:
i = 0
for i in range(0, len(data)):
    data[i]['data']['text'] = data[i]['data']['text'].replace('\t', '')

In [ ]:
for i in range(0, len(data)):
    j = 0
    for j in range(0, len(data[i]['annotations'][0]['result'])):
        if 'id' in data[i]['annotations'][0]['result'][j]:
            data[i]['annotations'][0]['result'][j].pop('id')
        if 'origin' in data[i]['annotations'][0]['result'][j]:
            data[i]['annotations'][0]['result'][j].pop('origin')
        if 'text' in data[i]['annotations'][0]['result'][j]:
            data[i]['annotations'][0]['result'][j].pop('text')


BLOCO ANTIGO MAIS EFETIVO ATÉ ENTÃO

In [ ]:
i = 0
j = 0
dataset1 = []
d1 = ()
for i in range(0, len(data)):
    for j in range(0, len(data[i]['annotations'][0]['result'])):
      if 'value' in data[i]['annotations'][0]['result'][j]:
        d1 = (data[i]['annotations'][0]['result'][j]['value']['start'], data[i]['annotations'][0]['result'][j]['value']['end'], data[i]['annotations'][0]['result'][j]['value']['labels'][0])
      else:
        pass
    dataset1.append((data[i]['data']['text'], [tuple(d1)]))

In [ ]:
random.shuffle(dataset1)

In [ ]:
TRAIN_DATA = dataset[:60]
DEV_DATA = dataset[60:]

In [ ]:
TRAIN_DATA = examples[:60]
DEV_DATA = examples[60:]

In [ ]:
db = DocBin()
db.to_disk("train.spacy", TRAIN_DATA)
db.to_disk("dev.spacy", DEV_DATA)

In [ ]:
TRAIN_DATA

In [ ]:
db = DocBin()
#dataset = dataset
i = 0
def convert(path, dataset):
    nlp = spacy.blank("pt")
    db = DocBin()
    #text = [dataset[i][0] for i in range(0, len(dataset))]
    #annot = [dataset[i][1] for i in range(0, len(dataset))]
    for i in range(0, len(dataset1)):
          doc = nlp_lg.make_doc(dataset1[i][0])
          ents = []
          stt = dataset1[i][1][0][0]
          end = dataset1[i][1][0][1]
          lbl = dataset1[i][1][0][2]
          span = doc.char_span(stt, end, label=lbl, alignment_mode="expand")
          ents.append(span)
          doc.ents = ents
          db.add(doc)
    db.to_disk(path)
convert("train.spacy", TRAIN_DATA)
convert("dev.spacy", DEV_DATA)

In [ ]:
!python -m spacy init config config.cfg --lang pt --pipeline ner --optimize efficiency --force

In [ ]:
!python -m spacy train config.cfg --output /content/output --paths.train ./train.spacy --paths.dev ./dev.spacy

In [ ]:
nlp_ner = spacy.load("/content/output/model-best")

In [ ]:
listDocs = []
docs = list(nlp_ner.pipe(listDocs))
for i in range(0, len(dfL['Conteúdo'])):
    art = nlp_ner(dfL['Conteúdo'][i])
    docs.append(art)
c_doc = Doc.from_docs(docs)

In [ ]:
for doc in docs:
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
dir(Example)

In [ ]:
listDocs = []
docs = list(nlp_ner.pipe(listDocs))
for i in range(0, len(dfLr['Conteúdo'])):
    art = nlp_ner(dfLr['Conteúdo'][i])
    docs.append(art)
c_doc = Doc.from_docs(docs)

          
  FIM TRECHO PROGRAMA EM USO A PARTIR DE 29/05/2024


In [ ]:
###ANTIGO. UTILIZADO NO EXAMPLES.FROM_DICT!!!###
i = 0
j = 0
dataset = []
d = {}
for i in range(0, len(data)):
    for j in range(0, len(data[i]['annotations'][0]['result'])):
      if 'value' in data[i]['annotations'][0]['result'][j]:
        d = {'entities': [(data[i]['annotations'][0]['result'][j]['value']['start'], data[i]['annotations'][0]['result'][j]['value']['end'], data[i]['annotations'][0]['result'][j]['value']['labels'][0])]}
      else:
        pass
    dataset.append([data[i]['data']['text'], d])

In [ ]:
for text, annotations in TRAIN_DATA:
    doc = nlp_lg(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        ents.append(span)
    doc.ents = ents
    db.add(doc)
    db.to_disk("./train.spacy")

In [ ]:
for text, annotations in DEV_DATA:
    doc = nlp_lg(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        ents.append(span)
    doc.ents = ents
    db.add(doc)
    db.to_disk("./dev.spacy")

In [ ]:
for text, annotations in DEV_DATA:
    doc = nlp_lg(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        ents.append(span)
        doc.ents = ents
    db.add(doc)
    db.to_disk("./dev.spacy")

In [ ]:
len(doc)

In [ ]:
with open("dataset1.json", "w", encoding='utf-8') as final:
    json.dump(dataset1, final, ensure_ascii=False)

In [ ]:
train(
    config_path=/content/base_config.cfg
    overrides=
)

In [ ]:
type(train)

In [ ]:
###NÃO FUNCIONOU!!!####
"""Convert entity annotation from spaCy v2 TRAIN_DATA format to spaCy v3
.spacy format."""
import srsly
import typer
import warnings
from pathlib import Path

import spacy
from spacy.tokens import DocBin


def convert(lang: str, input_path: Path, output_path: Path):
    nlp = spacy.blank(lang)
    db = DocBin()
    for text, annot in srsly.read_json(input_path):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is None:
                msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                warnings.warn(msg)
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(output_path)


'''if __name__ == "__main__":
    typer.run(convert)'''

In [ ]:
span = doc.char_span(dataset1[0][1][0][0], dataset1[0][1][0][1], label=dataset1[0][1][0][2], alignment_mode="strict")

In [ ]:
doc = nlp_lg.make_doc(dataset1[1][0])

In [ ]:
data

In [ ]:
#!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
len(DEV_DATA)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)